In [ ]:
from faker import Faker
import pandas as pd
import random
import os
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO, filename="add_1000_orders_logging.log",filemode="w")

payments= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv')
orders= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv')

merged_df=payments.merge(orders,on='payment_id',how='left',indicator=True)

payment_date_converted=pd.DatetimeIndex(merged_df['payment_date']) #convert from object to datetime payment_date_converte

merged_df['Year']=payment_date_converted.year
#merged_df['Quarter']=
merged_df['Month']=payment_date_converted.month  

merged_df.sort_values(by='payment_date',ascending=True,inplace=True)
merged_df.drop_duplicates(subset='payment_id',keep='first',inplace=True)#removing duplicates with same payment_id
columns_todrop=['customer_id_x','emp_id','created_at','payment_type', #removing columns that are not needed
                'order_id','item_id','customer_id_y','delivery',
                'delivery_address_id','item_quantity',
                'customer_modifications','delivery_notes','_merge']
merged_df.drop(columns=columns_todrop,axis=1,inplace=True)

columns=['cafe_id','payment_date','Year','Month','payment_id','amount']
merged_df=merged_df[columns]
merged_df['payment_date']=pd.to_datetime(merged_df['payment_date'])
merged_df['Year']=merged_df['Year'].astype('str')
merged_df['Month']=merged_df['Month'].astype('str')
#using .loc to change all values in column 'Month' to 2 digits
merged_df['Month'] = merged_df['Month'].str.zfill(2)
merged_df['Year_Month']=merged_df['Year']+'-'+merged_df['Month']

#rearranging columns and ordering by cafe_id and Year_Month
first_column = merged_df.pop('Year_Month') 
merged_df.insert(0, 'Year_Month', first_column)
first_column = merged_df.pop('cafe_id') 
merged_df.insert(0, 'cafe_id', first_column)
merged_df.sort_values(by=['cafe_id','Year_Month'],ascending=True,inplace=True)

#calculating total monthly revenue,monthly average check,monthly number of orders in each location 
cafe_locations_monthly_df=merged_df.groupby(['cafe_id','Year_Month','Year','Month']).agg(
    
    what_the_avg_check_was_this_month=('amount','mean'),
    number_of_orders_this_month=('payment_id','count'),
    monthly_revenue_total=('amount','sum')
).reset_index()
cafe_locations_monthly_df['what_the_avg_check_was_this_month']=pd.DataFrame(cafe_locations_monthly_df['what_the_avg_check_was_this_month'].apply(lambda x:round(x,2)))

#calculating  average revenue for each month of the year (Jan-Dec) in each location 
cafe_locations_monthly_avg_df=cafe_locations_monthly_df.groupby(['cafe_id','Month']).agg(
   avg_revenue_month_of_the_year_by_location=('monthly_revenue_total','mean')
).reset_index()
cafe_locations_monthly_avg_df['avg_revenue_month_of_the_year_by_location']=pd.DataFrame(cafe_locations_monthly_avg_df['avg_revenue_month_of_the_year_by_location'].apply(lambda x:round(x,2)))


#calculating  average monthly revenue in each location 
cafe_locations_avg_df=cafe_locations_monthly_df.groupby(['cafe_id']).agg(
   avg_monthly_revenue_by_location=('monthly_revenue_total','mean')
).reset_index()
cafe_locations_avg_df['avg_monthly_revenue_by_location']=pd.DataFrame(cafe_locations_avg_df['avg_monthly_revenue_by_location'].apply(lambda x:round(x,2)))

#joining all 3 dataframes 
cafe_locations_monthly_revenue_df=pd.merge(cafe_locations_monthly_df,cafe_locations_monthly_avg_df,how='left',on=['cafe_id','Month'])
cafe_locations_monthly_revenue_df.reset_index()
cafe_locations_df=pd.merge(cafe_locations_monthly_revenue_df,cafe_locations_avg_df,how='left',on=['cafe_id'])
cafe_locations_df.reset_index(drop=True,inplace=True)
cafe_locations_df.head()

#adding running total by month for each location (starts over in january of each year)
cafe_locations_df['Running_total_by_month']=pd.DataFrame(cafe_locations_df.groupby(['cafe_id','Year'])['monthly_revenue_total'].cumsum(
).reset_index(drop=True))

#downloading the dataframe as .csv file
cafe_locations_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Monthly_revenues.csv',index=False)

#exporting annual revenue for each cafe location in a separate .csv
Annual_revenue=pd.DataFrame(cafe_locations_df.groupby(['cafe_id','Year']).apply(lambda x: x['monthly_revenue_total'].sum())).reset_index()
Annual_revenue.reset_index(drop=True,inplace=True)
Annual_revenue.rename(columns={0:'Annual revenue'},inplace=True)
Annual_revenue.sort_values(by=['cafe_id','Year'])
Annual_revenue.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Annual_revenues.csv',index=False)


In [ ]:
#Creating separate files with monthly revenues for each of the locations:
cafe_1_df=cafe_locations_df[cafe_locations_df['cafe_id']==1]
cafe_1_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Cafe_1_monthly_revenues.csv',index=False)
cafe_2_df=cafe_locations_df[cafe_locations_df['cafe_id']==2]
cafe_2_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Cafe_2_monthly_revenues.csv',index=False)
cafe_3_df=cafe_locations_df[cafe_locations_df['cafe_id']==3]
cafe_3_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Cafe_3_monthly_revenues.csv',index=False)
cafe_4_df=cafe_locations_df[cafe_locations_df['cafe_id']==4]
cafe_4_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Cafe_4_monthly_revenues.csv',index=False)
cafe_5_df=cafe_locations_df[cafe_locations_df['cafe_id']==5]
cafe_5_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Cafe_5_monthly_revenues.csv',index=False)